# Exploration of data augmentation on Performance.
From [exploration](./3-ja-covnet-refinement.ipynb) we have found a structure:

**784 - 32C5-32C5S2-64C5-64C5S2- 128C4 - 10**

which performs reasonably well on validation data set (>99.4%).

Here we will test different data augmentation.


## Setup

In [2]:
import os
import sys
from pathlib import Path

sys.path.append("../")
from dotenv import find_dotenv, load_dotenv
from torch.utils.data import DataLoader
import torchvision.transforms as T
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard.writer import SummaryWriter

import logging

logger = logging.getLogger()
fhandler = logging.FileHandler(filename="mylog.log", mode="a")
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.INFO)
logger.info("Setting up logging...")


from src.utils import CustomMnistDataset, train_covnet

load_dotenv(find_dotenv())

DATA_DIR = os.getenv("DATA_DIR")
SEED = int(os.getenv("SEED"))  # type: ignore
TENSORBOARD_DIR = Path(os.getenv("TENSORBOARD_DIR"))
MODEL_DIR = Path(os.getenv("MODEL_DIR"))
MODEL_DIR.mkdir(parents=True, exist_ok=True)
BATCH_SIZE = 20
SHUFFLE = True
EPOCHS = 25
optimizer_wrapper = lambda x: optim.SGD(x, lr=0.01, momentum=0.90)

## Model
**784 - 32C5-32C5S2-64C5-64C5S2- 128C4 - 10** with Batch normalization and Dropout per layer with Test Time Augmentation.

In [3]:
class Net(nn.Module):
    def __init__(self, augmentor):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5) # 32, 24, 24, 
        self.pool1 = nn.Conv2d(32, 32, 5, 2, padding=2) 
        self.conv2 = nn.Conv2d(32, 64, 5) 
        self.pool2 = nn.Conv2d(64, 64, 5, 2, padding=2)
        self.conv3 = nn.Conv2d(64, 128, 4)
        self.bn3 = nn.BatchNorm2d(128)
        self.fc2 = nn.Linear(128, 10)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn1b = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn2b = nn.BatchNorm2d(64)
        self.d1 = nn.Dropout(p=0.1)
        self.d2 = nn.Dropout(p=0.25)
        self.d3 = nn.Dropout(p=0.4)
        self.augmentor = augmentor
        self.n_augs = 16
        

    def forward_model(self, x):
        # layer 1
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn1b(self.pool1(x)))
        x = self.d1(x) #(32, 12, 12)
        # layer 2
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn2b(self.pool2(x)))
        x = self.d2(x) #(64, 4, 4)
        # layer 3
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.d3(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc2(x)
        return x
    
    def forward(self, x):
        out = self.forward_model(x)
        if self.train:
            return out
        for _ in range(self.n_augs):
            out += self.forward_model(self.augmentor(x))
        return out / (self.n_augs + 1)


## Affine Augmentation.

In [3]:
exp_name = '32C5-32C5S2-D1-64C5-64C5S2-D2-128C4-D3-10-BN-TTA-AFFINE-SMALL'
augmentor = nn.Sequential(
    T.RandomAffine(degrees=5, translate=(0.05, 0.05), scale=(0.95, 1.05))
)
scripted_augmentor = torch.jit.script(augmentor) #type: ignore
train_dataset = CustomMnistDataset(
    img_dir=DATA_DIR, type="train", transform=scripted_augmentor
)
val_dataset = CustomMnistDataset(img_dir=DATA_DIR, type="validation")

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE)
train_dict = train_covnet(
    net = Net(scripted_augmentor),
    dataloader = train_dataloader,

    epochs = EPOCHS,
    optimizer_wrapper = optimizer_wrapper,
    criterion = nn.CrossEntropyLoss(),
    writer = SummaryWriter(Path(TENSORBOARD_DIR, exp_name)), #type; ignore
    val_dataloader = val_dataloader
)
torch.save(train_dict.get('model').state_dict(), Path(MODEL_DIR, exp_name) ) #type: ignore

In [4]:
exp_name = '32C5-32C5S2-D1-64C5-64C5S2-D2-128C4-D3-10-BN-TTA-AFFINE-LARGE'
augmentor = nn.Sequential(
    T.RandomAffine(degrees=22.5, translate=(0.2, 0.2), scale=(0.8, 1.2))
)
scripted_augmentor = torch.jit.script(augmentor) #type: ignore
train_dataset = CustomMnistDataset(
    img_dir=DATA_DIR, type="train", transform=scripted_augmentor
)
val_dataset = CustomMnistDataset(img_dir=DATA_DIR, type="validation")

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE)
train_dict = train_covnet(
    net = Net(scripted_augmentor),
    dataloader = train_dataloader,

    epochs = EPOCHS,
    optimizer_wrapper = optimizer_wrapper,
    criterion = nn.CrossEntropyLoss(),
    writer = SummaryWriter(Path(TENSORBOARD_DIR, exp_name)), #type; ignore
    val_dataloader = val_dataloader
)
torch.save(train_dict.get('model').state_dict(), Path(MODEL_DIR, exp_name) ) #type: ignore

## Elastic

In [6]:
exp_name = '32C5-32C5S2-D1-64C5-64C5S2-D2-128C4-D3-10-BN-TTA-ELASTIC-SMALL'
augmentor = nn.Sequential(
    T.ElasticTransform(alpha=50.0, sigma=5.0)
)
scripted_augmentor = augmentor #type: ignore
train_dataset = CustomMnistDataset(
    img_dir=DATA_DIR, type="train", transform=scripted_augmentor
)
val_dataset = CustomMnistDataset(img_dir=DATA_DIR, type="validation")

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE)
train_dict = train_covnet(
    net = Net(scripted_augmentor),
    dataloader = train_dataloader,

    epochs = EPOCHS,
    optimizer_wrapper = optimizer_wrapper,
    criterion = nn.CrossEntropyLoss(),
    writer = SummaryWriter(Path(TENSORBOARD_DIR, exp_name)), #type; ignore
    val_dataloader = val_dataloader
)
torch.save(train_dict.get('model').state_dict(), Path(MODEL_DIR, exp_name) ) #type: ignore

In [10]:
exp_name = '32C5-32C5S2-D1-64C5-64C5S2-D2-128C4-D3-10-BN-TTA-ELASTIC-LARGE'
augmentor = nn.Sequential(
    T.RandomAffine(degrees=, translate=(0.2, 0.2), scale=(0.8, 1.2)),
    T.ElasticTransform(alpha=100.0, sigma=5.0)
)
scripted_augmentor = augmentor #type: ignore
train_dataset = CustomMnistDataset(
    img_dir=DATA_DIR, type="train", transform=scripted_augmentor
)
val_dataset = CustomMnistDataset(img_dir=DATA_DIR, type="validation")

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE)
train_dict = train_covnet(
    net = Net(scripted_augmentor),
    dataloader = train_dataloader,

    epochs = EPOCHS,
    optimizer_wrapper = optimizer_wrapper,
    criterion = nn.CrossEntropyLoss(),
    writer = SummaryWriter(Path(TENSORBOARD_DIR, exp_name)), #type; ignore
    val_dataloader = val_dataloader
)
torch.save(train_dict.get('model').state_dict(), Path(MODEL_DIR, exp_name) ) #type: ignore

## Affine + Elastic

In [4]:
exp_name = '32C5-32C5S2-D1-64C5-64C5S2-D2-128C4-D3-10-BN-TTA-ELASTIC-AFFINE-SMALL'
augmentor = nn.Sequential(
    T.RandomAffine(degrees=5, translate=(0.05, 0.05), scale=(0.95, 1.05)),
    T.ElasticTransform(alpha=30.0, sigma=5.0)
)
scripted_augmentor = augmentor #type: ignore
train_dataset = CustomMnistDataset(
    img_dir=DATA_DIR, type="train", transform=scripted_augmentor
)
val_dataset = CustomMnistDataset(img_dir=DATA_DIR, type="validation")

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE)
train_dict = train_covnet(
    net = Net(scripted_augmentor),
    dataloader = train_dataloader,

    epochs = EPOCHS,
    optimizer_wrapper = optimizer_wrapper,
    criterion = nn.CrossEntropyLoss(),
    writer = SummaryWriter(Path(TENSORBOARD_DIR, exp_name)), #type; ignore
    val_dataloader = val_dataloader
)
torch.save(train_dict.get('model').state_dict(), Path(MODEL_DIR, exp_name) ) #type: ignore

# Prediction

In [24]:
model = Net(augmentor)
model.load_state_dict(torch.load(Path(MODEL_DIR, exp_name)))
model.eval()
preds = []
test_dataset = CustomMnistDataset(img_dir=DATA_DIR, type="test")
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
for inputs, in iter(test_dataloader):
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    preds.extend(predicted.tolist())

In [32]:
import pandas as pd
prediction_df = pd.DataFrame({"ImageId": range(1,len(preds)+1), "Label": preds})
prediction_df.to_csv(Path(DATA_DIR) / ('../'+exp_name +'_submission.csv' ), index=False)